<a href="https://colab.research.google.com/github/RaRagi-Bliss/Python-Text-Analyzer/blob/imp-LING226-final-asgn/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary links and documents for this assignment

text data
https://drive.google.com/drive/folders/11IlS87sebzk6D02Ks61MNKbAOu6aKw08?usp=sharing

report
https://docs.google.com/document/d/1LTluiE6PhXJilCgR17c1JrCsZ9U9MhrdsQ585GhWR78/edit?usp=sreport

spreadsheet data
https://docs.google.com/spreadsheets/d/1DL2CJ3KbLecQjci2Kc6raacRLCMXVFRs4bEr8ugySBQ/edit?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import
import nltk # import the main nltk module
import re # import regular expression
import csv # import spreadsheets
import os
import warnings
from nltk.corpus import stopwords #import stop words
from nltk.corpus import cmudict
from nltk.corpus import words
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import SyllableTokenizer
from nltk.probability import FreqDist
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt_tab')  # previously nltk.download('punkt'), but has since
                            # been deprecated, replaced with ('punkt_tab')
nltk.download('words')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('cmudict')

sid = SentimentIntensityAnalyzer()
cmu = cmudict.dict()

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
#Preprocessing
def remove_punctuation(text):
    if isinstance(text, str):  # Check if input is a string
        return re.sub(r'[^\w\s]', '', text)  # Remove all punctuation except for underscores, words, and spaces
    else:
        raise ValueError("Input should be a string")

def remove_stopWords(text):
  text_tokens = text #word_tokenize(text)
  stoppedWords = [word for word in text_tokens if not word in stopwords.words('english')]
  output = ''.join(text)
  return output

#broken
def filterNonWords(text):
    # Tokenize the lyrics into words
    tokens = nltk.word_tokenize(text.lower())   # Convert to lowercase for consistency
    english_words = set(words.words())            # Get a set of words from the NLTK words corpus
    filtered_words = [word for word in tokens if word in english_words]     # Filter words that are in the English dictionary
    filtered_lyrics = ' '.join(filtered_words)    # Join the filtered words back into a string
    return filtered_lyrics

#broken
def removeShortWords(text):
    tokens = word_tokenize(text) # Tokenize the text into words
    filtered_tokens = [word for word in tokens if len(word) > 3]  # Filter out words that are three characters or shorter
    filtered_text = ''.join(filtered_tokens)     # Join the remaining words back into a string

    return filtered_text

def preprocess(text):
  input = text
  one = remove_punctuation(input)
  output = remove_stopWords(one)
  #output = filterNonWords(two) broken
  #output = removeShortWords(three) broken
  return output



In [5]:
#analysis

def totalWords(text):
  return len(nltk.word_tokenize(text))

def totalUniqueWords(text):
  return len(set(nltk.word_tokenize(text)))

def countSyllables(text):
  # Initialize a SyllableTokenizer instance
  syllable_tokenizer = SyllableTokenizer()
  # Tokenize the text into syllables
  syllables = syllable_tokenizer.tokenize(text)
  return len(syllables)

def countSentences(text):
  sentences = text.split('\n')
  return len(sentences)

def lexicalDiversity(text):
  print(f"Total number of words: \t{totalWords(text)}")
  print(f"Total unique words: \t{len(set(nltk.word_tokenize(text.lower())))}")

  ld = len(set(nltk.word_tokenize(text)))/len(nltk.word_tokenize(text))

  print(f"Total repeated words: \t{totalWords(text) - len(set(nltk.word_tokenize(text.lower())))}")
  print(f"Lexical Diversity of Text: {ld}")

  sentences = sent_tokenize(text)
  total_sentence_ld = 0

  for sentence in sentences:
    sentence_words = word_tokenize(sentence.lower())
    sentence_unique_words = set(sentence_words)

    sentence_words_count = len(sentence_words)
    sentence_unique_words_count = len(sentence_unique_words)

    if sentence_words_count > 0:
      sentence_ld = sentence_unique_words_count / sentence_words_count
      total_sentence_ld += sentence_ld

  if len(sentences) > 0:
    average_sentence_ld = total_sentence_ld / len(sentences)
    print(f"Average Lexical Diversity per sentence: {average_sentence_ld}")
  else:
    print("No sentences found in the text.")

def wordFrequencies(text):
  input = text.lower()
  text_tokens = input.split(" ")
  text_fdist = FreqDist(text_tokens)
  print(f"Most Common Words: {text_fdist.most_common(10)}")

def count_syllables_v2(word):

    if word in cmu.keys():
      # get the entry from the dictionary, and slice the first set of sounds
      phones = cmu[word][0]

      # using a list comprehension, extract only the sounds that end in a digit
      vowel_sounds= [sound for sound in phones if sound[-1].isdigit()]

      # the number of syllabes is equal to the number of vowel_sounds,
      syllables = len(vowel_sounds)

    else:
      # If the word is not found in the dictionary, use a simple rule
      # based on the number of vowel letters
      vowels = "aeiouy"
      syllables = 0
      prev_char_is_vowel = False
      for char in word:
          if char in vowels:
              if not prev_char_is_vowel:
                  syllables += 1
              prev_char_is_vowel = True
          else:
              prev_char_is_vowel = False

    #print(f'The word {word} has {syllables} syllables.')
    return syllables

def text_info(text):
  """
  Args:
    text: a string
  Returns:
    ...
  """
  # lowercase the text
  text = text.lower()

  # extract tokens, removing any that are just punctuation
  tokens = [token.lower() for token in nltk.word_tokenize(text) if token.isalpha()]

  # extract sentences
  sentences = countSentences(text)

  # extract syllables
  syllables = 0

  for token in tokens:
    syllables += count_syllables_v2(token)
  print(f'This text has {len(tokens)} words, {sentences} sentences, \nand {syllables} syllables.')
  return len(tokens), sentences, syllables

def fleschReadingEase(text):
  words, sents, sylls = text_info(text)

  word_sents = words/sents
  syll_words = sylls/words

  reading_ease_score = 206.835 - (1.015 * word_sents) - (84.6 * syll_words)

  print(f'Flesch Reading Ease Score: {reading_ease_score}')

def sentiment_lookup(text):
  #calculates average sentiment
  tokens = nltk.word_tokenize(text.lower())
  output = []

  for token in tokens:
    # first check if the token is even in the dictionary
    if token in sid.lexicon.keys():
      # add the rating to the output
      output.append(sid.lexicon[token])

 # we calculate the average sentiment of the text from all the values in our output
  if output:
    avg_sentiment = sum(output)/len(output)
  # how many words from the sentence were actually used in the calculation
    print(f'Sentiment is: {avg_sentiment},\nusing {round(len(output)/len(tokens)*100, 2)}% of the words in the text,\nwhich is {len(output)} words total')
  else:
    print('Nothing in this text was in the sentiment dictionary')

##Implement saving to file(Book 17)



In [6]:
#Showing Data
def textInformation(text):

  with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, module="nltk")
    print ("---------------------------------")
    print ("\t TEXT INFORMATION \n")
    print (f"Total number of words: \t{totalWords(text)}")
    print (f"Total unique words: \t{totalUniqueWords(text)}")
    print (f"Number of Syllables:  \t{countSyllables(text)}")
    print (f"Number of Sentences:  \t{countSentences(text)}")
    print ("\n---------------------------------")
    print ("\t TEXT SENTIMENT \n")
    sentiment_lookup(text)
    print ("\n---------------------------------")
    print ("\t FLESCH READING \n")
    fleschReadingEase(text)
    print ("\n---------------------------------")
    print ("\t LEXICAL DIVERSITY \n")
    lexicalDiversity(text)
    print ("\n---------------------------------")
    print ("\t TOP 10 WORDS")
    wordFrequencies(text)
    print ("\n---------------------------------")


In [7]:
#Art Museum
folder_path = "/content/drive/MyDrive/LING226/Art Museum (Musee d'Orsay)"

In [8]:
#History Museum
folder_path = "/content/drive/MyDrive/LING226/History Museum (Abbey Library of Saint Gall)"

In [9]:
#National Museum
folder_path = "/content/drive/MyDrive/LING226/National Museum (Zurich Museum)"

In [10]:
#National Museum
folder_path = "/content/drive/MyDrive/LING226/National Museum (Te Papa)"

In [ ]:

# List files in the folder
file_list = os.listdir(folder_path)

# Display the list of files
print("Files in 'text_files' folder:")
for file_name in file_list:
    print(file_name)

for file_name in file_list:
    full_path = os.path.join(folder_path, file_name)
    if file_name.endswith('.txt'):  # Check if it's a text file
        with open(full_path, 'r') as file:
            content = file.read()
            print(f"Contents on {file_name}:")
            textInformation(preprocess(content))
            doc = nlp(content)
            noun_phrases = [chunk.text for chunk in doc.noun_chunks]
            print(f'Noun Phrases in {file_name}: \n {noun_phrases}')